In [5]:
"""
Adapted from: https://github.com/aliasvishnu/EEGNet/blob/master/EEGNet-PyTorch.ipynb

Original paper - https://arxiv.org/abs/1611.08024

Please reach out to me if you spot an error
"""

'\nAdapted from: https://github.com/aliasvishnu/EEGNet/blob/master/EEGNet-PyTorch.ipynb\n\nOriginal paper - https://arxiv.org/abs/1611.08024\n\nPlease reach out to me if you spot an error\n'

In [6]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [7]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 64), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        
    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*7)
        x = F.sigmoid(self.fc1(x))
        return x

In [10]:
net = EEGNet().cuda(0)
print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.7925]], device='cuda:0', grad_fn=<SigmoidBackward>)


**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [26]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [12]:
X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 64).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 64).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))

**Run**

In [27]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, X_train, y_train, params))
    print ("Validation - ", evaluate(net, X_val, y_val, params))
    print ("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.2934, device='cuda:0')
Train -  [0.56, 0.5034343434343435, 0.6393442622950819]
Validation -  [0.4, 0.4436363636363636, 0.42307692307692313]
Test -  [0.42, 0.41874999999999996, 0.4528301886792453]

Epoch  1
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.5117, device='cuda:0')
Train -  [0.46, 0.48282828282828283, 0.5344827586206896]
Validation -  [0.56, 0.5612121212121213, 0.6]
Test -  [0.48, 0.5170833333333333, 0.46938775510204084]

Epoch  2
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3808, device='cuda:0')
Train -  [0.54, 0.5442424242424242, 0.5660377358490566]
Validation -  [0.47, 0.4602020202020202, 0.4175824175824176]
Test -  [0.51, 0.53375, 0.4948453608247421]

Epoch  3
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.4416, device='cuda:0')
Train -  [0.58, 0.5931313131313131, 0.6037735849056604]
Validation -  [0.49, 0.4488888888888889, 0.4631578947368421]
Test -  [0.47, 0.5116666666666667, 0.4536082474226804]

/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages

Test -  [0.49, 0.4804166666666666, 0.4950495049504951]

Epoch  6
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3224, device='cuda:0')
Train -  [0.48, 0.5163636363636364, 0.5806451612903226]
Validation -  [0.53, 0.5442424242424243, 0.5913043478260869]
Test -  [0.54, 0.5633333333333334, 0.5306122448979592]

Epoch  7
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3310, device='cuda:0')
Train -  [0.64, 0.572929292929293, 0.7230769230769231]
Validation -  [0.51, 0.5260606060606061, 0.5585585585585585]
Test -  [0.44, 0.48625, 0.48148148148148157]

Epoch  8
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3258, device='cuda:0')
Train -  [0.52, 0.5959595959595959, 0.6000000000000001]
Validation -  [0.52, 0.5305050505050505, 0.5932203389830508]
Test -  [0.45, 0.4358333333333333, 0.4660194174757281]

Epoch  9
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.2558, device='cuda:0')
Train -  [0.69, 0.6824242424242425, 0.7438016528925621]
Validation -  [0.47, 0.45131313131313133, 0.5309

/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/cs/home/ybk1/python/lib/python3.7/site-packages